In [1]:
import math
from sklearn import neighbors
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [2]:
def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    X = []
    y = []
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            print("doing1")
            continue
        #print("doing")
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)
    #print(X.shape())
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

In [24]:
def find(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.6):
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)
    print(X_img.shape)
    if len(X_face_locations) == 0:
        return []
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


In [4]:
def show_prediction_labels_on_image(img_path, predictions):
    
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        name = name.encode("UTF-8")

        
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    del draw

    pil_image.show()
    


In [23]:
if __name__ == "__main__":
    train_path = '/home/sreemahavishnu/Desktop/programming/courses/udacity/deep_learning/facial_recognition/10k_small/faces/'
    test_path = '/home/sreemahavishnu/Desktop/programming/courses/udacity/deep_learning/facial_recognition/faces/test_45/'
    print("Training KNN classifier...")
    #classifier = train(train_path, model_save_path="trained_knn_model.clf", n_neighbors=2)
    print("Training complete!")
    count = 0
    matched1 = []
    unmatched_ori1 = []
    unmatched_exp1 = []
    for image_file in os.listdir(test_path):
        full_file_path = os.path.join(test_path, image_file)

        print("Looking for faces in {}".format(image_file))

        predictions = find(full_file_path, model_path="trained_knn_model.clf")


        for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))
            if name == image_file[:4] :
                matched1.append(name[2:4])
                count += 1
            else :
                unmatched_ori1.append(name[2:4])
                unmatched_exp1.append(image_file[2:4])

        show_prediction_labels_on_image(os.path.join(test_path, image_file), predictions)
print(count/len(os.listdir(test_path)))
matched1.sort()
unmatched_ori1.sort()
unmatched_exp1.sort()

Training KNN classifier...
Training complete!
Looking for faces in A_05.jpg
(480, 640, 3)
- Found A_05 at (315, 118)
Looking for faces in A_88.jpg
(480, 640, 3)
- Found A_88 at (366, 180)
Looking for faces in A_41.jpg
(480, 640, 3)
- Found A_41 at (407, 201)
Looking for faces in A_25.jpg
(480, 640, 3)
- Found A_25 at (315, 167)
Looking for faces in A_90.jpg
(480, 640, 3)
- Found A_90 at (340, 118)
Looking for faces in A_50.jpg
(480, 640, 3)
- Found A_50 at (390, 219)
Looking for faces in A_29.jpg
(480, 640, 3)
- Found A_29 at (340, 167)
Looking for faces in A_12.jpg
(480, 640, 3)
- Found A_12 at (319, 142)
Looking for faces in A_66.jpg
(480, 640, 3)
- Found A_66 at (345, 180)
Looking for faces in A_89.jpg
(480, 640, 3)
- Found A_89 at (345, 98)
Looking for faces in A_51.jpg
(480, 640, 3)
- Found A_51 at (315, 192)
Looking for faces in A_10.jpg
(480, 640, 3)
- Found A_10 at (260, 112)
Looking for faces in A_04.jpg
(480, 640, 3)
- Found A_06 at (365, 167)
Looking for faces in A_69.jpg
(4

In [10]:
print(matched1, unmatched_ori1, unmatched_exp1)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '46', '47', '48', '49', '50', '51', '52', '53', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90'] ['53', '71'] ['45', '54']


In [14]:
print(matched1, unmatched_ori1, unmatched_exp1)

['04', '07', '10', '13', '15', '16', '17', '18', '20', '22', '23', '24', '25', '27', '28', '30', '31', '32', '33', '36', '38', '39', '40', '43', '44', '49', '52', '53', '54', '55', '56', '57', '59', '60', '62', '63', '66', '68', '69', '70', '71', '72', '73', '74', '76', '78', '79', '81', '82', '83', '84', '86', '87', '88'] ['17', '18', '20', '20', '22', '23', '24', '24', '25', '25', '25', '25', '30', '31', '33', '33', '38', '38', '43', '52', '52', '54', '56', '59', '59', '62', '63', '66', '66', '70', '71', '76', '78', '83', '84', '87'] ['01', '02', '03', '05', '06', '08', '09', '11', '12', '14', '19', '21', '26', '29', '34', '35', '37', '41', '42', '45', '46', '47', '48', '50', '51', '58', '61', '64', '65', '67', '75', '77', '80', '85', '89', '90']


In [26]:
if __name__ == "__main__":
    train_path = '/home/sreemahavishnu/Desktop/programming/courses/udacity/deep_learning/facial_recognition/10k_small/faces/'
    test_path = '/home/sreemahavishnu/Desktop/programming/courses/udacity/deep_learning/facial_recognition/faces/test6/'
    print("Training KNN classifier...")
    classifier = train(train_path, model_save_path="trained_knn_model.clf", n_neighbors=2)
    print("Training complete!")
    count = 0
    matched1 = []
    unmatched_ori1 = []
    unmatched_exp1 = []
    for image_file in os.listdir(test_path):
        full_file_path = os.path.join(test_path, image_file)

        print("Looking for faces in {}".format(image_file))

        predictions = predict(full_file_path, model_path="trained_knn_model.clf")


        for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))
            if name == image_file[:4] :
                matched1.append(name[2:4])
                count += 1
            else :
                unmatched_ori1.append(name[2:4])
                unmatched_exp1.append(image_file[2:4])

show_prediction_labels_on_image(os.path.join(test_path, image_file), predictions)
print(count/len(os.listdir(test_path)))
matched1.sort()
unmatched_ori1.sort()
unmatched_exp1.sort()

Training KNN classifier...
Training complete!
Looking for faces in A_05.jpg
- Found A_05 at (315, 118)
Looking for faces in A_88.jpg
Looking for faces in A_41.jpg
- Found A_66 at (428, 201)
Looking for faces in A_25.jpg
- Found A_25 at (340, 167)
Looking for faces in A_90.jpg
- Found A_90 at (340, 118)
Looking for faces in A_50.jpg
- Found A_66 at (411, 211)
Looking for faces in A_29.jpg
- Found A_29 at (365, 167)
Looking for faces in A_12.jpg
- Found A_12 at (319, 142)
Looking for faces in A_66.jpg
- Found A_66 at (345, 180)
Looking for faces in A_89.jpg
- Found A_70 at (340, 68)
Looking for faces in A_51.jpg
- Found A_51 at (345, 222)
Looking for faces in A_10.jpg
- Found A_10 at (260, 112)
Looking for faces in A_04.jpg
- Found A_04 at (390, 167)
Looking for faces in A_69.jpg
- Found A_40 at (366, 139)
Looking for faces in A_30.jpg
- Found A_30 at (340, 167)
Looking for faces in A_38.jpg
- Found A_38 at (324, 180)
Looking for faces in A_68.jpg
- Found A_40 at (340, 192)
Looking for f

In [61]:
if __name__ == "__main__":
    train_path = '/home/sreemahavishnu/Desktop/programming/courses/udacity/deep_learning/facial_recognition/10k_small/faces/'
    test_path = '/home/sreemahavishnu/Desktop/programming/courses/udacity/deep_learning/facial_recognition/faces/test3/'
    print("Training KNN classifier...")
    classifier = train(train_path, model_save_path="trained_knn_model.clf", n_neighbors=2)
    print("Training complete!")
    count = 0
    matched = []
    unmatched_ori = []
    unmatched_exp = []
    for image_file in os.listdir(test_path):
        full_file_path = os.path.join(test_path, image_file)

        print("Looking for faces in {}".format(image_file))

        predictions = predict(full_file_path, model_path="trained_knn_model.clf")
        print(full)

        for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))
            if name == image_file[:4] :
                matched.append(name)
                count += 1
            else :
                unmatched_ori.append(name)
                unmatched_exp.append(image_file)

show_prediction_labels_on_image(os.path.join(test_path, image_file), predictions)
print(count/len(os.listdir(test_path)))
print(matched, unmatched_ori, unmatched_exp)

Training KNN classifier...
Training complete!
Looking for faces in A_05.jpg
- Found A_54 at (315, 118)
Looking for faces in A_88.jpg
Looking for faces in A_41.jpg
- Found A_66 at (242, 201)
Looking for faces in A_25.jpg
Looking for faces in A_90.jpg
Looking for faces in A_50.jpg
- Found A_01 at (159, 201)
Looking for faces in A_29.jpg
- Found A_70 at (117, 167)
Looking for faces in A_12.jpg
- Found A_38 at (18, 142)
Looking for faces in A_66.jpg
- Found A_66 at (97, 180)
Looking for faces in A_89.jpg
Looking for faces in A_51.jpg
- Found A_24 at (283, 222)
Looking for faces in A_10.jpg
- Found A_10 at (260, 112)
Looking for faces in A_04.jpg
- Found A_04 at (171, 142)
Looking for faces in A_69.jpg
- Found A_67 at (159, 139)
Looking for faces in A_30.jpg
- Found A_01 at (43, 167)
Looking for faces in A_38.jpg
- Found A_38 at (159, 180)
Looking for faces in A_68.jpg
- Found A_67 at (365, 192)
Looking for faces in A_82.jpg
Looking for faces in A_37.jpg
Looking for faces in A_55.jpg
- Foun

In [62]:
l = []
for i, j in zip(unmatched_ori, unmatched_exp) :
    if i[2:4] == j[2:4] :
        l.append(i[2:4])
print(len(l) + len(matched))

24


In [63]:
print(len(matched), len(unmatched_ori), len(unmatched_exp))

24 40 40


In [15]:
    from sklearn.externals import joblib
    joblib.dump(classifier, 'model.pkl')

['model.pkl']